# HPK

In [1]:
#Split file (1 file contains 14 MaPSAs data)
import re
with open('HPKSheet1.csv', 'r') as file:
    raw = file.read()
    raw = raw.strip(raw[0:115])

rows = raw.split('\n')
delim = str(rows[1].split(',')[0].split('_')[0]) #use 35494 as delimeter
sections = re.split(delim, raw)

#filename_list = []
#for section in sections:
#    lines = section.strip().split('\n')
#    data = delim + section
#    filename = data.split(',')[0]
#    filename_list.append(filename)
#del filename_list[0]
#print(filename_list)

In [2]:
import re
import pandas as pd
import xml.etree.ElementTree as ET
import xml.dom.minidom


#functions
def getHPKdata(filename):
    chip_data = pd.read_csv(filename+".csv", names = ['number', 'location','WaffelPackno','WaffelPackrow','WaffelPackcol', 'WaferID','Waferrow','Wafercol','BINcode'])
    return chip_data

def getMapsaName(filename):
    return 'HPK_'+filename[0:9]+filename[-1:]

def HPK_getPosn(num):
    if num>8:
        Posn=25-num
    else:
        Posn=num
    return Posn

def HPK_getNameLabel(chip_data, no):
    name = chip_data['WaferID'][no][:6] + '_'+  chip_data['WaferID'][no][6:8]
    row = abs(chip_data['Waferrow'][no])
    col = chip_data['Wafercol'][no]
    coor = str(int(col)-9).zfill(2)
    if int(coor) < 0:
        col = abs(int(coor))
        coor =str(1)+str(col)
    return str(name)+'_'+str(row)+coor

#HPK split
def HPK_split(sheetname):
    with open(sheetname, 'r') as file:
        raw = file.read()
        raw = raw.strip(raw[0:115])
    rows = raw.split('\n')
    delim = str(rows[1].split(',')[0].split('_')[0]) #use 35494 as delimeter
    sections = re.split(delim, raw)
    sections.pop(0)
    filename_list = []

    for section in sections:
        data = delim+section
        filename = data.split(',')[0]
        filename_list.append(filename)
        with open(filename + '.csv', 'w') as output_file:
            output_file.write(data)
    return filename_list

#for HPK sheet 
def HPK_getXML(sheetname):
    filename_list = HPK_split(sheetname) #split,save csv files
    for filename in filename_list:
        chip_data = getHPKdata(filename)
       
        root = ET.Element("ROOT")
        parts = ET.SubElement(root, "PARTS")

        # MaPSA block
        MAPSA = ET.SubElement(parts, "PART", mode="auto") #use ElementTree to add mode='auto'
        ET.SubElement(MAPSA, "KIND_OF_PART").text = "MaPSA"
        ET.SubElement(MAPSA, "NAME_LABEL").text = getMapsaName(filename)
        ET.SubElement(MAPSA, "MANUFACTURER").text = "Hamamatsu"
        ET.SubElement(MAPSA, "LOCATION").text = "Hamamatsu"
        ET.SubElement(MAPSA, "VERSION").text = "2.0"
    
        predefMapsa1 = ET.SubElement(MAPSA, "PREDEFINED_ATTRIBUTES")
        attr1 = ET.SubElement(predefMapsa1, "ATTRIBUTE")
        ET.SubElement(attr1, "NAME").text = "Status"
        ET.SubElement(attr1, "VALUE").text = "Good"
        child = ET.SubElement(MAPSA, "CHILDREN")
    
        df = getHPKdata(filename)
        for index, row in df.iterrows():
            child_sub = ET.SubElement(child, "PART", mode="auto")
            ET.SubElement(child_sub, "KIND_OF_PART").text = "MPA Chip"
            ET.SubElement(child_sub, "NAME_LABEL").text = str(HPK_getNameLabel(chip_data, index))
            #Predefined attributes of child
            child_predef = ET.SubElement(child_sub, "PREDEFINED_ATTRIBUTES")
            child_predef_attr = ET.SubElement(child_predef, "ATTRIBUTE")
            ET.SubElement(child_predef_attr, "NAME").text = "Chip Posn on Sensor"
            ET.SubElement(child_predef_attr, "VALUE").text = str(HPK_getPosn(row['location']))

    
        sensor_sub = ET.SubElement(child, "PART", mode="auto")
        ET.SubElement(sensor_sub, "KIND_OF_PART").text = "PS-p Sensor"
        ET.SubElement(sensor_sub, "NAME_LABEL").text = chip_data['number'][0]
        xmlstr = ET.tostring(root)
        dom = xml.dom.minidom.parseString(xmlstr)
        xmlfinal = dom.toprettyxml(indent="   ")
        
        ET.indent(ET.ElementTree(root),'   ')
        f = open(getMapsaName(filename)+'.xml', "wb")
        ET.ElementTree(root).write(f)
        print(getMapsaName(filename))
    return 

In [3]:
HPK_getXML('HPKSheet1.csv')

HPK_35494_032L
HPK_35494_033L
HPK_35494_036L
HPK_35494_037L
HPK_35494_038L
HPK_35494_039L
HPK_35494_040L
HPK_35494_041L
HPK_35494_042L
HPK_35494_043L
HPK_35494_044L
HPK_35494_046L
HPK_35494_047L
HPK_35494_048L
